In [1]:
# Auto Reload
%load_ext autoreload
%autoreload 2

In [32]:
# Import Modules
import numpy as np
import copy
import pandas as pd
import json
import requests
from bs4 import BeautifulSoup
from pprint import pprint
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import re
import os
import time

In [25]:
url = 'https://www.sephora.com/product/protini-tm-polypeptide-cream-P427421'

In [85]:
chrome_options = Options()  
# chrome_options.add_argument("--headless")
driver = webdriver.Chrome(options=chrome_options)
driver.get(url)
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
time.sleep(1)
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
time.sleep(2)
page = driver.page_source
driver.close()
driver.quit()

In [86]:
html = BeautifulSoup(page, 'html.parser')

In [87]:
z = html.findAll('div', {'data-sephid':'5'})

In [88]:
stars = html.find("div", {'data-comp':"StarRating "})['aria-label']
stars

'4 stars'

In [89]:
review_score = html.find("div", {'data-comp':"ReviewsStats Box "})
review_score.div

<div class="css-1g66t8c" data-comp="Flex Box "><span class="css-tc6qfq" data-comp="Text Box ">4386 reviews</span><button class="css-k13wt5" data-comp="Link Box " type="button">Write a review</button></div>

In [91]:
# Parse for item features
def sephora_parse_item(item_url:str) -> dict:
    '''Returns item/product features from a given sephora url'''
    # GET Request to URL
    page = requests.get(item_url)
    # Create BS4 Object
    html = BeautifulSoup(page.text, "html.parser")
    
    # Find Name and Brand
    brand, name = (x.get_text() for x in html.find("h1", attrs={"data-comp":True}))
    
    # Find n_loves (product likes)
    n_loves = html.find('span', attrs={"data-at":"product_love_count"}).get_text()
    
    return brand, name, n_loves

In [92]:
brand, name, n_loves = sephora_parse_item('https://www.sephora.com/product/protini-tm-polypeptide-cream-P427421')

brand, name, n_loves

('Drunk Elephant', 'Protini™ Polypeptide Moisturizer', '259023')

In [56]:
def sephora_scrape(url:str) -> dict:
    '''Returns dictionary of needed features parsed from sephora.com link using selenium for dynamic page loading'''
    # Init Chrome Driver Options
    chrome_options = Options()  
    # Run headless arg to avoid GUI
    chrome_options.add_argument("--headless")
    # Init Driver Instance -> load options
    driver = webdriver.Chrome(options=chrome_options)
    # Call url via driver
    driver.get(url)
    # Scroll down to load REVEIW section
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(5)
    page = driver.page_source
    driver.close()
    driver.quit()